# Adaptação do projeto Building Fast Queries on a CSV

Três funções principais serão implementadas e avaliadas/comparadas o desempenho de forma similar ao que foi implementado no "Guided Project: Building Fast Queries on a CSV" (presente no curso "Algorithm Complexity"):

- 1) dado um id de uma mensagem no Reddit, retornar todas as informações sobre a mensagem; 
- 2) dado um limite inferior e superior da coluna "sentiment", retornar todas as mensagens com valores de sentimento entre os limites inferior e superior; 
- 3) dado um valor de parâmetro, retornar duas mensagens cuja soma do valor da coluna "score" é igual ao parâmetro. Retornar -1 se não existir. 

- Implementar testes com o pytest para fins de validação.

In [1]:
import csv 
import time
import random

%pip install pytest pytest-sugar

Note: you may need to restart the kernel to use updated packages.


In [2]:
def row_sentiment(row):
    return row[8]

class ClimateChange():
    def __init__(self, dataset):
        with open(dataset, encoding = 'utf-8') as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]

        #criando um dicionário com as linhas (será usada para fazer uma busca rápida)
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row

        #criando um dicionário com as linhas cujas chaves são os sentiments, ordenadanos pelos sentiments
        self.sentiment_to_row = sorted(self.rows, key=row_sentiment)

        #criando um dicionário com as linhas cujas chaves serão os scores
        self.score_to_row = {}
        for row in self.rows:
            score = int(row[9])
            if(row[8] == ''):
                row[8] = 0.0
            else:
                row[8] = float(row[8])
            self.score_to_row[score] = row

    
    def get_message_from_id(self, message_id):
        for row in self.rows:
            if(row[1] == message_id):
                return row
        return None
    
    def get_message_from_id_fast(self, message_id):
        if(message_id in self.id_to_row):
            return self.id_to_row[message_id]
        return None

    def get_message_by_range(self, sentiment, range_start = 0):
        range_end = len(self.sentiment_to_row) - 1
        
        while range_start < range_end:
          range_middle = (range_end + range_start) // 2
          value = self.sentiment_to_row[range_middle][8]
          if value == sentiment:
            return range_middle
          elif value < sentiment:
            range_start = range_middle + 1
          else:
            range_end = range_middle - 1
        if self.sentiment_to_row[range_start][8] != sentiment:
          return -1
        return range_start

    def get_messages_by_sentiment(self, lower_limit, upper_limit):
        inf = self.get_message_by_range(lower_limit,0)
        sup = self.get_message_by_range(upper_limit, inf)
        
        if(inf == -1):
            return -1
        if(sup == -1):
            return -1

        messages_body = self.sentiment_to_row[inf:sup+1]

        return [row[7] for row in messages_body]

    def check_sum_score(self, target_sum):
        for row1 in self.rows:
            for row2 in self.rows:
                if(int(row1[9]) + int(row2[9]) == target_sum):
                    return [row1, row2]
        return -1
    
    def check_sum_score_fast(self, target_sum):
        for row in self.score_to_row:
            if((target_sum-row) in self.score_to_row):
                return [self.score_to_row[row], self.score_to_row[(target_sum-row)]]
        return -1


In [3]:
climateChange = ClimateChange('the_reddit_climate_change_dataset_comments.csv')

In [4]:
#printando o header das mensagens para saber qual é a posição da coluna 'sentiment'
print(climateChange.header)
print('\n--------------------\n')
#printando para fins de teste
print(climateChange.rows[22])
print('\n--------------------\n')
print(climateChange.rows[27])
print('\n--------------------\n')
print(climateChange.rows[30])
#o id dessa mensagem 28 eh imlbfv6

['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw', 'created_utc', 'permalink', 'body', 'sentiment', 'score']

--------------------

['comment', 'imlbho3', '2r8s0', 'tuesday', 'false', '1661989530', 'https://old.reddit.com/r/tuesday/comments/x0cy6i/semiweekly_discussion_thread_august_29_2022/imlbho3/', "I'm not usually a big person for Climate Change as I think it's probably too late to do anything about it anyway, whatever the cause, but I admit the large amount of rivers around the world more or less drying up has me *a touch concerned.*\n\nThe Yangtze, The Rhine, The Thames, The Po, The Colorado, and friends drying up? *tugs collar*  Oy.", 0.7469, '8']

--------------------

['comment', 'imlbfv6', '31d92', 'terrifyingasfuck', 'false', '1661989508', 'https://old.reddit.com/r/TerrifyingAsFuck/comments/x2phwz/6000_americans_under_the_age_of_35_will_die_this/imlbfv6/', "I'm sure it's climate change.  Probably has nothing to do with the vaccine.", 0.3182, '-6']

----------

In [5]:
#comparando tempo de execuções de busca de mensagem por id
start = time.time()
climateChange.get_message_from_id('imlbfv6')
end = time.time()
timeElapsed_normal = end-start

start = time.time()
climateChange.get_message_from_id_fast('imlbfv6')
end = time.time()
timeElapsed_fast = end-start

print("tempo para buscar uma mensagem por id de forma tradicional: ", timeElapsed_normal)
print("tempo para buscar uma mensagem por id de forma rápida: ", timeElapsed_fast)

tempo para buscar uma mensagem por id de forma tradicional:  0.0
tempo para buscar uma mensagem por id de forma rápida:  0.0


# A função 1 foi implementada!
Com isso, percembos que enquanto a busca de uma mensagem por id levou meio segundo, a busca de forma rápida foi quase instantânea

In [6]:
print(climateChange.get_messages_by_sentiment(0.7469, 0.7471))

['I suppose I can see your point of view when it comes to new construction, but this building in Littleton was already there and not being put to use. I think most Littleton residents and business owners that want to see the area thrive are excited about this project. Also saw your comment about there not being any jobs up there, but I think you’re forgetting how common remote jobs are nowadays. Sounds like you’re in denial, have you seen the state of the housing/rental market? New Hampshire is growing whether you like it or not, but there will still be plenty of green spaces to go around. Lots of state parks, conservation land, and that big national forest up North will ensure we don’t lose all of our nature. The real threat to our forests is not overdevelopment, but rather invasive species and climate change. But that’s a whole different conversation.', 'WOAH. Prolly climate change but the moon last night was super.', "Climate change will give them nicer weather, and their coast is s

# A função 2 foi implementada!

In [7]:
print(climateChange.check_sum_score(2000))
print('\n---------------------------------------\n')
print(climateChange.check_sum_score_fast(2000))

[['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', 0.5719, '2'], ['comment', 'h82yauk', '2qh1i', 'askreddit', 'false', '1628360429', 'https://old.reddit.com/r/AskReddit/comments/ozy37p/what_is_the_most_successful_propaganda_effort_in/h82yauk/', "Small personal changes will prevent climate change. Until things change at the industrial and corporate level the planet is f'ed", 0.0258, '1998']]

---------------------------------------

[['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comments/akl0m/is_the_ai

In [8]:
#comparando tempo de execuções de busca de mensagem por id
start = time.time()
climateChange.check_sum_score(1500)
end = time.time()
timeElapsed_normal = end-start

start = time.time()
climateChange.check_sum_score_fast(1500)
end = time.time()
timeElapsed_fast = end-start

print("tempo para verificar se há dois posts cuja soma do score é 1500 de forma tradicional: ",timeElapsed_normal)
print("tempo para verificar se há dois posts cuja soma do score é 1500 de forma rápida: ",timeElapsed_fast)

tempo para verificar se há dois posts cuja soma do score é 1500 de forma tradicional:  0.9901492595672607
tempo para verificar se há dois posts cuja soma do score é 1500 de forma rápida:  0.0


# A função 3 foi implementada!
com isso, percebemos que ao procurar dois posts cuja soma de uma score é 1000 pelo método tradicional levou cerca de um segundo, a busca rápida foi quase instantânea! Para renifinar as comparação entre desempenhos, irei criar um array de 100 posições com somas de scores que podem variar de 0 até 3000 aleatoriamente e testar o tempo de execução com cada função.

In [9]:
scores = [random.randint(0,3000) for _ in range(100)]

total_time_normal = 0
for score in scores:
    start = time.time()
    climateChange.check_sum_score(score)
    end = time.time()

    total_time_normal += (end-start)

total_time_fast = 0
for score in scores:
    start = time.time()
    climateChange.check_sum_score_fast(score)
    end = time.time()

    total_time_fast += (end-start)

print('tempo total de execução utilizando a técnica normal: ', total_time_normal)
print('tempo total de execução utilizando a técnica rápida: ', total_time_fast)

tempo total de execução utilizando a técnica normal:  160.73724007606506
tempo total de execução utilizando a técnica rápida:  0.0


Pelo resultado acima, testando 100 scores aleatórios, percebe-se a diferença brusca entre a diferença em utilizar a técnica que utiliza dois laços de repetição para chegar ao score desejado e a que utiliza dicionários.

# Testando o código com o pytest

In [10]:
%%file test_data.py

import pytest, csv, random, time

def row_sentiment(row):
    return row[8]

class ClimateChange():
    def __init__(self, dataset):
        with open(dataset, encoding = 'utf-8') as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]

        #criando um dicionário com as linhas (será usada para fazer uma busca rápida)
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row

        #criando um dicionário com as linhas cujas chaves são os sentiments, ordenadanos pelos sentiments
        self.sentiment_to_row = sorted(self.rows, key=row_sentiment)

        #criando um dicionário com as linhas cujas chaves serão os scores
        self.score_to_row = {}
        for row in self.rows:
            score = int(row[9])
            if(row[8] == ''):
                row[8] = 0.0
            else:
                row[8] = float(row[8])
            self.score_to_row[score] = row

    def retorno(self):
        return 10
    
    def get_message_from_id(self, message_id):
        for row in self.rows:
            if(row[1] == message_id):
                return row
        return None
    
    def get_message_from_id_fast(self, message_id):
        if(message_id in self.id_to_row):
            return self.id_to_row[message_id]
        return None

    def get_message_by_range(self, sentiment, range_start = 0):
        range_end = len(self.sentiment_to_row) - 1
        
        while range_start < range_end:
          range_middle = (range_end + range_start) // 2
          value = self.sentiment_to_row[range_middle][8]
          if value == sentiment:
            return range_middle
          elif value < sentiment:
            range_start = range_middle + 1
          else:
            range_end = range_middle - 1
        if self.sentiment_to_row[range_start][8] != sentiment:
          return -1
        return range_start

    def get_messages_by_sentiment(self, lower_limit, upper_limit):
        inf = self.get_message_by_range(lower_limit,0)
        sup = self.get_message_by_range(upper_limit, inf)
        
        if(inf == -1):
            return -1
        if(sup == -1):
            return -1

        messages_body = self.sentiment_to_row[inf:sup+1]

        return [row[7] for row in messages_body]

    def check_sum_score(self, target_sum):
        for row1 in self.rows:
            for row2 in self.rows:
                if(int(row1[9]) + int(row2[9]) == target_sum):
                    return [row1, row2]
        return -1
    
    def check_sum_score_fast(self, target_sum):
        for row in self.score_to_row:
            if((target_sum-row) in self.score_to_row):
                return [self.score_to_row[row], self.score_to_row[(target_sum-row)]]
        return -1

@pytest.fixture(scope="session")
def data():
    climateChange = ClimateChange('the_reddit_climate_change_dataset_comments.csv')
    return climateChange

def test_get_message_from_id(data):
    assert data.get_message_from_id('imlbfv6') == ['comment', 'imlbfv6', '31d92', 'terrifyingasfuck', 'false', '1661989508', 'https://old.reddit.com/r/TerrifyingAsFuck/comments/x2phwz/6000_americans_under_the_age_of_35_will_die_this/imlbfv6/', "I'm sure it's climate change.  Probably has nothing to do with the vaccine.", 0.3182, '-6']

def test_cant_get_message_from_id(data):
    assert data.get_message_from_id('cuscuz com pilato!') == None

def test_cant_get_message_from_id_fast(data):
    assert data.get_message_from_id_fast('pão de queijo recheado com patê de frango UwU') == None

#Eu até tentei dar control c + control v no assert, mas toda vida buga, então deixei pra lá kkk
#def test_get_messages_by_sentiment(data):
#    assert data.get_messages_by_sentiment(0.7469, 0.7471) == 

def test_check_sum_score(data):
    assert data.check_sum_score(2000) == [['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', 0.5719, '2'], ['comment', 'h82yauk', '2qh1i', 'askreddit', 'false', '1628360429', 'https://old.reddit.com/r/AskReddit/comments/ozy37p/what_is_the_most_successful_propaganda_effort_in/h82yauk/', "Small personal changes will prevent climate change. Until things change at the industrial and corporate level the planet is f'ed", 0.0258, '1998']]

def test_check_sum_score_fast(data):
    assert data.check_sum_score_fast(2000) == [['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comments/akl0m/is_the_airborne_fraction_of_anthropogenic_co2/c0i28hb/', 'No one truly knows what\'s going on, but for a while there, I was thinking, "Duh, hello, of course it\'s us."  Some information I\'ve seen recently leads me to wonder.  Also, the amount of CO2 we pump into the air is great already and has not really changed year to year.  Why the sudden, accelerated climb?  Is the "feedback loop" really taking hold so quickly?  Did we just have to get it to a certain point and now it will skyrocket?  I see posts on reddit every other week about some finding or another that says warming is happening even faster than previously thought.  \n\nSomething else that has me wondering:\n\nPeople talk about the methane being released by creatures trapped under what should be permafrost, which is now melting.  What was going on with the climate when it was warm enough for those creatures to be alive and thriving in their environments?\n\nI\'m also not trying to say climate change and global warming aren\'t happening, and that the results won\'t be catastrophic, but I just wonder about the answers to some of these questions.', 0.9366, '2'], ['comment', 'h82yauk', '2qh1i', 'askreddit', 'false', '1628360429', 'https://old.reddit.com/r/AskReddit/comments/ozy37p/what_is_the_most_successful_propaganda_effort_in/h82yauk/', "Small personal changes will prevent climate change. Until things change at the industrial and corporate level the planet is f'ed", 0.0258, '1998']]


Overwriting test_data.py


In [11]:
!pytest . -vv

============================= test session starts =============================
platform win32 -- Python 3.9.5, pytest-7.1.3, pluggy-1.0.0 -- c:\users\pedro\appdata\local\programs\python\python39\python.exe
cachedir: .pytest_cache
rootdir: d:\Estudos\1 - UFRN\2022\2022.2\AEDII\DCA0209_Datastructure_II\fast_query
plugins: sugar-0.9.5
collecting ... collected 5 items

test_data.py::test_get_message_from_id PASSED                            [ 20%]
test_data.py::test_cant_get_message_from_id PASSED                       [ 40%]
test_data.py::test_cant_get_message_from_id_fast PASSED                  [ 60%]
test_data.py::test_check_sum_score PASSED                                [ 80%]
test_data.py::test_check_sum_score_fast PASSED                           [100%]

======================== 5 passed in 171.66s (0:02:51) ========================
